In [1]:
import os
import shutil
import pandas as pd
import re
import binascii # 내장모듈
import pickle # 내장모듈
from os.path import join
from tqdm import tqdm
import traceback
from upload_v2 import write_log_csv

### 중복관리 코드

In [ ]:
def sort_dupl(pathOfDict:str, nameOfDict:str) :

    with open(join(pathOfDict, nameOfDict), "rb") as pkl :
        dict = pickle.load(pkl)
    
    total = len(dict)
    
    for __key__, ListOfSubDict in tqdm(dict.items(), total=total) : # 파일 그룹 단위 반복
    #"score" : score, "root" : root, "stem":stem, "ext":ext, "fullPath" : fullPath, "size":size, "crc32": crc32, "mtime":mtime, "sell":sell
        num = len(ListOfSubDict)
        if num == 1 :
            pass
        else :
            for i in range(num):
                for j in range((i+1), num) :
                    if ListOfSubDict[i]["score"] < ListOfSubDict[j]["score"] : # 바뀌는 경우
                        ListOfSubDict[i], ListOfSubDict[j] = ListOfSubDict[j], ListOfSubDict[i] # 딕셔너리 내부에서도 작동함.

    with open(join(pathOfDict,("정렬된 "+nameOfDict)), "wb") as pkl :
        pickle.dump(dict, pkl)

def readDictOnebyOne(pathOfDict:str, nameOfDict:str) :
    with open(join(pathOfDict, nameOfDict), "rb") as pkl :
        dict = pickle.load(pkl)
    total = len(dict)
    print(f"전체 그룹 수는 {total}")

    for key, list in dict.items() :
        print(key)
        templist = []
        for x in list : 
            templist.append(x["score"])
        print("sore  : ", *templist, sep="  ")
        print("마지막 파일 pathOfDict", x["fullPath"])
        print("더 읽을까 말까? y or n")
        go = input()
        if go == "y" :
            pass
        else :
            break

In [ ]:
path = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\중복조사"
name = '정렬된 sc별 파일정보.pkl'

In [ ]:
sort_dupl(path, name)

In [ ]:
readDictOnebyOne(path, name)

In [ ]:
def changeDictForNas(pathOfDict, nameOfDict) :
    with open(join(pathOfDict, nameOfDict), "rb") as pkl :
        dict = pickle.load(pkl)
    total = len(dict)

    for key, list in tqdm(dict.items(), total=total) :
        for x in list : 
            x["root"] = re.sub(r"D:\\", "/volume1/스캔파일/", x["root"])
            x["root"] = re.sub(r"\\", "/", x["root"])
            x["fullPath"] = re.sub(r"D:\\", "/volume1/스캔파일/", x["fullPath"])
            x["fullPath"] = re.sub(r"\\", "/", x["fullPath"])

    with open(join(pathOfDict,("nas용 "+nameOfDict)), "wb") as pkl :
        pickle.dump(dict, pkl)

In [ ]:
path = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\중복조사"
name = '정렬된 sc별 파일정보.pkl'
changeDictForNas(path, name)

In [ ]:
def moveDuplFile(pathOfDict, nameOfDict, basePathToMove) : 
    with open(join(pathOfDict, nameOfDict), "rb") as pkl :
        dict = pickle.load(pkl)
    total = len(dict)
    print(f"{total}개의 그룹에 대해 중복 파일 이동을 실시합니다.")
    dupl_fail = []
    duplCnt = 0
    
    for __key__, list in tqdm(dict.items(), total=total) :
        num = len(list)
        if num > 1 : # 중복파일
            for i in range(1, num) :
                try :
                    src = list[i]["fullPath"]
                    #dir = os.path.split(src)[0]
                    file = os.path.split(src)[1]

                    dst_dir = join(basePathToMove, file) # 대표파일이름으로 폴더 만들어 중복파일을 넣는다
                    dst = join(dst_dir, file)

                    if not os.path.exists(dst_dir) :
                        os.makedirs(dst_dir)
                    shutil.move(src, dst)   
                    duplCnt += 1

                except Exception as e :
                    dupl_fail.append([src, dst, e.__class__, e])
                    continue  # 코드 계속 진행

    print(f"중복파일 이동 : {duplCnt}건,  이동실패 : {len(dupl_fail)}건")
    write_log_csv(dupl_fail, pathOfDict)



In [ ]:
pathOfDict = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\중복조사"
nameOfDict = '정렬된 sc별 파일정보.pkl'
basePathToMove = r'D:\구 스캔파일_dupl'
moveDuplFile(pathOfDict, nameOfDict, basePathToMove)

In [ ]:
def moveKeyFile(pathOfDict, nameOfDict, basePathToMove, path_gu) :
    with open(join(pathOfDict, nameOfDict), "rb") as pkl :
        dict = pickle.load(pkl)
    total = len(dict)
    gu = len(path_gu) + 1
    print(f"{total}개의 그룹에 대해 key가 있는 파일 이동을 실시합니다.")
    noKeyList = []
    move_fail = []
    keyCnt, extraKeyCnt = 0, 0

    for __key__, list in tqdm(dict.items(), total=total) :
        try :
            # 1) 점수가 가장 높고, key 있는 파일 채무자키폴더로 이동
            # root - path -1(디렉토리 구분자 제거) = subdir
            if list[0]["score"] >= 100000 :
                src = list[0]["fullPath"]
                #dir = os.path.split(src)[0]
                file = os.path.split(src)[1]

                dst_dir = join(basePathToMove, file) # 대표파일이름으로 폴더 만들어 중복파일을 넣는다
                dst = join(dst_dir, file)

                dst_dir = join(basePathToMove, "key", list[0]["root"][gu:])
                dst = join(dst_dir, file)

                if not os.path.exists(dst_dir) :
                    os.makedirs(dst_dir)
                shutil.move(src, dst)
                keyCnt += 1

            # 2) 점수가 가장 높지만, extraKey 있는 파일 extraKey폴더로 이동
            elif list[0]["score"] >= 10000 :
                src = list[0]["fullPath"]
                #dir = os.path.split(src)[0]
                file = os.path.split(src)[1]

                dst_dir = join(basePathToMove, "extraKey", list[0]["root"][gu:])
                dst = join(dst_dir, file)

                if not os.path.exists(dst_dir) :
                    os.makedirs(dst_dir)
                shutil.move(src, dst)
                extraKeyCnt += 1

            # 점수가 가장 높지만, key가 전혀 없는 것 root, stem, ext, fullPath(\\192...으로 변경해야 하니까 원형 유지를 위해 root, fileName도 저장한다.)
            # 를 df에 담아 for문 끝난 후 엑셀로 저장
            # fullPath 하이퍼 링크 걸어주고, 편하게 파일 보면서 newName 저장
            else :
                noKeyList.append([list[0]["root"], list[0]["stem"], list[0]["ext"], list[0]["fullPath"]])
        
        except Exception as e :
            move_fail.append([src, dst, e.__class__, e])
            continue  # 코드 계속 진행
    
    print(f"채무자키 : {keyCnt}건,  기타키 : {extraKeyCnt}건,  이동실패 : {len(move_fail)}건")
    write_log_csv(move_fail, pathOfDict)
    
    # 제 자리에 두는 파일 목록 엑셀로 변환
    df_noKey = pd.DataFrame(noKeyList, columns=["root", "stem", "ext", "fullPath"])
    df_noKey.to_excel(join(pathOfDict, "remain.xlsx"))

In [ ]:
pathOfDict = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\중복조사"
nameOfDict = '정렬된 sc별 파일정보.pkl'
basePathToMove = r'D:\\구 스캔파일_'
path_gu = r'D:\구 스캔파일'
moveKeyFile(pathOfDict, nameOfDict, basePathToMove, path_gu)

### 서버에 이미 올라간 파일, 0byte, xps류 파일 이동하기

In [3]:
# 서버 dict불러오기
path_kcs = r'\\192.168.0.75/스캔파일/스캔파일log/_project/파일/중복조사/kcs별 파일정보_솔림헬프.pkl'
def read_cks(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)
dict_kcs = read_cks(path_kcs)

In [4]:
len(dict_kcs)

128022

In [5]:
set_keys = set()
for k in dict_kcs.keys() :
    set_keys.add(k[8:])
len(set_keys)
# kcs = 127903개 , cs = 126278

126397

In [6]:
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

In [3]:
# 폴더 리스트 만들기
pathList = [r"D:\extraKey", r"D:\key", r"D:\구 스캔파일", r"D:\구 스캔파일_key"]
zeroDir = r"D:\구 스캔파일_exception\0byte"
xpsDir = r"D:\구 스캔파일_exception\xps"
serverDir = r"D:\구 스캔파일_exception\서버등록"


In [4]:
#server = []
#serverFail = []
xps = []
xpsFail = []
p = re.compile("xps", re.I)
for path in pathList :
    for root, __dirs__, files in tqdm(os.walk(path)) :
        for f in files :
            fullPath = join(root, f)
            # size = os.path.getsize(fullPath)
            # if size == 0 :
            #     shutil.move(fullPath, join(zeroDir, f))
            # else :
            #     crc = crc32_checksum(fullPath)
            #     cs = str(crc) + str(size)
            #     if cs in set_keys :
            #         dst_dir = join(serverDir, cs)
            #         dst = join(dst_dir, f)
            #         if not os.path.exists(dst_dir) :
            #             os.mkdir(dst_dir)
            #         try :
            #             os.rename(fullPath, dst)
            #             server.append([fullPath, dst])
            #         except Exception as e:
            #             serverFail.append([fullPath, dst, e.__class__])
            #             pass
                        
            #     else :
            ext = os.path.splitext(f)[1]

            if p.search(ext) :
                dst = join(xpsDir, f)
                try :
                    os.rename(fullPath, dst)
                    xps.append([fullPath, dst])
                except Exception as e:
                    xpsFail.append([fullPath, dst, e.__class__])
                    pass

#write_log_csv(server, serverDir)
write_log_csv(xps, xpsDir)
#write_log_csv(serverFail, serverDir)
write_log_csv(xpsFail, xpsDir)

17530it [00:55, 316.27it/s] 
1600it [00:08, 181.90it/s]
99967it [05:07, 324.93it/s] 
89it [00:00, 2976.75it/s]

log 파일이 비어있습니다. path : D:\구 스캔파일_exception\xps


### 빈 폴더 삭제

In [13]:
import os
os.chdir("c:/")
os.getcwd()

pathList = [r"D:\extraKey", r"D:\key", r"D:\구 스캔파일", r"D:\구 스캔파일_key"]
cnt = 0
for path in pathList :
    for root, dirs, files in tqdm(os.walk(path)) :
        for dir in dirs :
            try :
                os.removedirs(os.path.join(root, dir))
                cnt += 1
            except : # 파일 있는 경우
                pass

print(f"{cnt}개의 빈 디렉토리 삭제")

17142it [00:03, 5079.11it/s]
1557it [00:00, 3827.79it/s]
38860it [01:07, 576.82it/s] 
88it [00:00, 4414.74it/s]

61539개의 빈 디렉토리 삭제
